In [ ]:
import requests
from random import uniform
import time
import numpy as np
import re
from bs4 import BeautifulSoup as BS
from pymongo import MongoClient
from pymongo.errors import *

occupation = 'data-scientist'


client = MongoClient('localhost', 27017)
db = client['hh_vacancies']
collection = db.data_scientist

url = 'https://hh.ru'
suffix = '/vacancies/' + occupation
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0'}
info = []
last_page = False
page = 0

while True:
  params = {'page': str(page)}
  time.sleep(uniform(1, 3))
  response = requests.get(url + suffix, params=params, headers=headers)
  dom = BS(response.text, 'html.parser')
  vacancies = dom.find_all('div', {'class':'vacancy-serp-item-body__main-info'})
  vacancies_list = []
  for vacancy in vacancies:
    vacancy_data = {}
    vac_info = vacancy.contents[0]
    link_name = vac_info.find('h3').contents[0].contents[0]
    vacancy_data['name'] = link_name.string
    vacancy_data['link'] = link_name.find('a')['href']
    try:
      salary = vac_info.find('span', {'class':'bloko-header-section-3'}).text
      s = re.findall('\d+', salary.replace('\u202f', ''))
      if 'от' in salary:
        vacancy_data['salary_from']= int(s[0])
        vacancy_data['salary_to'] = np.nan
        vacancy_data['salary_currency'] = salary[salary.rfind(' '):]
      elif 'до' in salary:
        vacancy_data['salary_from'] = np.nan
        vacancy_data['salary_to'] = int(s[0])
        vacancy_data['salary_currency'] = salary[salary.rfind(' '):]
      else:
        vacancy_data['salary_from'] = int(s[0])
        vacancy_data['salary_to'] = int(s[1])
        vacancy_data['salary_currency'] = salary[salary.rfind(' '):]
    except:
      vacancy_data['salary_from'] = np.nan
      vacancy_data['salary_to'] = np.nan
      vacancy_data['salary_currency'] = np.nan

    vacancy_data['_id'] = vacancy_data['link']

    try:
      collection.insert_one(vacancy_data)
    except:
      pass

  if dom.find('div', {'class':'pager'}).find('a', {'data-qa' : 'pager-next'}) != None:
    page += 1
  else:
    break

In [ ]:
salary = 120000

res = collection.find({'$or' : [
    {'salary_from':{'$gt':salary}},
    {'salary_to':{'$gt':salary}}]})

print(res)